In [1]:
%load_ext autoreload
%autoreload 2
from dataset import SpikingDataset
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
#import seaborn as sns
import sys
sys.path.append('../../')
from dataset import Dataset, SpikingDataset, RegSpikingDataset, NewDataset
from torch.utils.data.dataloader import DataLoader
from Loss import KDLoss
import torch
import torch.nn.functional as F
import torch.nn as nn
import torchvision
random.seed(1338)
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import copy
from pyESN import ESN
from scipy import interpolate
from gen_data import *
from tanh import tanh
import pandas as pd

from Loss import ber_loss
from preprocessing import *


import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader

# import slayer from lava-dl
import lava.lib.dl.slayer as slayer

import IPython.display as display
from matplotlib import animation

generating_data = False

%load_ext autoreload

%autoreload 2

torch.__version__
# The coarse network structure is dicated by the Fashion MNIST dataset. 
dtype = torch.float

# Check whether a GPU is available
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    #device = torch.device("cuda")     
else:
    device = torch.device("cpu")
print(device)
tau_mem = 10e-3
tau_syn = 5e-3
time_step = 1e-3
alpha   = float(np.exp(-time_step/tau_syn))
beta    = float(np.exp(-time_step/tau_mem))

weight_scale = 7*(1.0-beta) # this should give us some spikes to begin with

print("init done")
print(device)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
cpu
init done
cpu


In [6]:
class Network(torch.nn.Module):
    def __init__(self, input_size, output_size, med_size, 
                 input_thres, input_tau_grad, input_scale_grad, 
                 thres, tau_grad, scale_grad):
        super(Network, self).__init__()

       # neuron_params = {
       #         'threshold'     : 0.1,
       #         'current_decay' : 1,
       #         'voltage_decay' : 0.1,
       #         'requires_grad' : True,     
       #     }
        #neuron_params_drop = {**neuron_params, 'dropout' : slayer.neuron.Dropout(p=0.05),}
        neuron_params = {
                'threshold'     : input_thres,
                'current_decay' : input_tau_grad,
                'voltage_decay' : input_scale_grad,
                'tau_grad'      : tau_grad,
                'scale_grad'    : scale_grad,
                'requires_grad' : True,  
                # 'graded_spike'  : True,
            
            }
        
        neuron_input_params = {
                'threshold'     : -10.0,
                'current_decay' : 0.25,
                'voltage_decay' : 0.03,
                'tau_grad'      : 0.03,
                'scale_grad'    : 1e-1,
                'requires_grad' : False,  
                # 'graded_spike'  : True,
            
            }
        
        neuron_params_2 = {
                'threshold'     : 0.05,    # delta unit threshold
                'tau_grad'      : 0.5,    # delta unit surrogate gradient relaxation parameter
                'scale_grad'    : False,      # delta unit surrogate gradient scale parameter
                'requires_grad' : True,   # trainable threshold
                'shared_param'  : True,   # layer wise threshold
                'period'        : 32,
                'decay'         : 0.25,
                'graded_spike'  : True,
                'norm' : slayer.neuron.norm.MeanOnlyBatchNorm,
            
        }
        sdnn_params = { # sigma-delta neuron parameters
                'threshold'     : thres,    # delta unit threshold
                'tau_grad'      : tau_grad,    # delta unit surrogate gradient relaxation parameter
                'scale_grad'    : scale_grad,      # delta unit surrogate gradient scale parameter
                'requires_grad' : True,   # trainable threshold
                'shared_param'  : True,   # layer wise threshold
                'activation'    : F.relu, # activation function
            }
        
        sdnn_input_params = { # sigma-delta neuron parameters
                'threshold'     : input_thres,    # delta unit threshold
                'tau_grad'      : input_tau_grad,    # delta unit surrogate gradient relaxation parameter
                'scale_grad'    : input_scale_grad,      # delta unit surrogate gradient scale parameter
                'requires_grad' : True,   # trainable threshold
                'shared_param'  : True,   # layer wise threshold
                'activation'    : F.relu, # activation function
            }
        
        
        self.blocks = torch.nn.ModuleList([
                #nn.Linear(input_size * 4, 32),
                #nn.ReLU(),
                #nn.Linear(32, output_size)
                #slayer.block.cuba.Input(neuron_params),
                # slayer.block.cuba.Recurrent(neuron_params, input_size, 32, weight_norm=True, delay=False, weight_scale=2.0),
                #slayer.block.rf.Recurrent(neuron_params_2, input_size, 32, weight_norm=True, delay=False, weight_scale=2.0),
                #slayer.block.cuba.Conv(neuron_params,  input_size, 24, 3, padding=1, stride=2, weight_scale=2, weight_norm=True),
                #slayer.block.rf.Dense(neuron_params=neuron_params_2, in_neurons=input_size, out_neurons=32, weight_norm=True, delay=True, weight_scale=2.0),
                #slayer.block.sigma_delta.Input(sdnn_input_params), 
                #slayer.block.sigma_delta.Dense(neuron_params=sdnn_params, in_neurons=input_size, out_neurons=med_size, weight_norm=True, delay=False, weight_scale=2.0),
                #slayer.block.sigma_delta.Output(sdnn_params, in_neurons=64, out_neurons=32, weight_norm=True, weight_scale=2.0),
                #slayer.block.sigma_delta.Dense(neuron_params=sdnn_params, in_neurons=64, out_neurons=128, weight_norm=True, delay=False, weight_scale=2.0),
                # slayer.block.cuba.Input(neuron_input_params), 
                #slayer.block.cuba.Input(neuron_input_params),
                #slayer.block.cuba.Recurrent(neuron_recurrent_params, input_size, med_size, weight_norm=True, delay=False, weight_scale=2.0),
                slayer.block.cuba.Dense(neuron_params=neuron_params, in_neurons=input_size, out_neurons=med_size, weight_norm=True, delay=True, weight_scale=2.0),
                #slayer.block.cuba.Dense(neuron_params=neuron_params, in_neurons=32, out_neurons=64, weight_norm=True, delay=True, weight_scale=2.0),
                #slayer.synapse.complex.Dense(64, 32, weight_norm=True),
                slayer.synapse.complex.Dense(med_size, 1, weight_norm=True),
                # slayer.synapse.complex.Affine(neuron_params, 32, 2, weight_norm=True, weight_scale=2.0),
                # slayer.synapse.complex.Dense(neuron_params, input_size, 32, weight_norm=True, delay=True, weight_scale=2.0),
                # slayer.block.cuba.Affine(neuron_params, 32, 2, weight_norm=True, weight_scale=2.0),
            ])
        
    
    def forward(self, spike):
        # spike = torch.flatten(spike, start_dim=1)
        for block in self.blocks:
            spike = block(spike)
        #spike = spike.flatten(start_dim=1)
        #x = x.flatten(start_dim=1)
        #spike = torch.cat((x, spike), dim=1)
        #real = self.fc_real(spike)
        #imag = self.fc_imag(spike)
        return spike

    def export_hdf5(self, filename):
        # network export to hdf5 format
        h = h5py.File(filename, 'w')
        layer = h.create_group('layer')
        for i, b in enumerate(self.blocks):
            b.export_hdf5(layer.create_group(f'{i}'))


In [7]:
def train_epoch(train_loader, net_snn, optimizer, scheduler, loss_fn):
    all_output = []
    inputs = []
    labels = []
    
    for X, y in train_loader:
        X = X.to(device).float()
        y = y.to(device).float()
        
        re, imag = net_snn(X)
        re = torch.mean(re, dim=2)
        imag = torch.mean(imag, dim=2)
        output = torch.cat((re, imag), axis=1)
        
        loss = loss_fn(output, y) 
        loss.backward()
        optimizer.step()
        
    scheduler.step()

          

def test(test_loader, net_snn, rc, num_frame = 19):
    all_output = []
    inputs = []
    labels = []
    
    for X, y in test_loader:
        X = X.to(device)
        y = y.to(device)
        
        #output = net_snn(X)
        #output = torch.mean(output, dim=2)
        re, imag = net_snn(X)
        re = torch.mean(re, dim=2)
        imag = torch.mean(imag, dim=2)
        output = torch.cat((re.view(-1, 1), imag.view(-1, 1)), axis=1)
    
        all_output.append(output.cpu().detach().numpy())
        labels.append(y.cpu().detach().numpy())
        
    all_output = np.concatenate(all_output, axis=0)
    labels = np.concatenate(labels, axis=0)
    predict_time = rc.time_to_freq(all_output, num_frame, remove_delay=False)
    target_time = rc.time_to_freq(labels, num_frame, remove_delay=False)
    # print(predict_time, target_time)
    return rc.my_new_test(predict_time, target_time)

def train(train_loader, test_loader, net_snn, epochs=100, rc=None, num_frame = 19):
    optimizer = torch.optim.Adam(net_snn.parameters(), lr=5e-3, betas=(0.9,0.999))
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
    #loss_fn = torch.nn.L1Loss()
    loss_fn = torch.nn.MSELoss()
    best_acc = 1e6
    for i in range(epochs):
        train_epoch(train_loader, net_snn, optimizer, scheduler, loss_fn)
        acc = test(test_loader, net_snn, rc, num_frame=19)
        best_acc = min(best_acc, acc)
        
    return best_acc

In [9]:
silent = True
method = 'RLS'  # RLS; INV; INV+RLS
# N_total_frame = 17
N_total_frame = 94
N_sync_frame = 4
# SNR_list = np.arange(1,20,2)
SNR_list = [10]
nb_inputs = 6

# Dataset selection
folder_name = 'data/S2/'  # LOS_Near:S2, LOS_Far:S3, NLOS:S1
output_folder = 'data_outputs/S1'

if folder_name == 'data/S1/':  # NLOS
    delay = 0
    packet_num = 21
elif folder_name == 'data/S2/':  # LOS_Near
    delay = 1
    packet_num = 27 # correct
elif folder_name == 'data/S3/':  # LOS_Far
    delay = 1
    packet_num = 22 # 23
else:
    print("Undefined Dataset")
    exit(1)
    
window_size = 2
N_reservoir = 16
debug = False

ber_record = []
dfe_ber_record = []
LS_ber_record = []
comb_ber_record = []
sta_ber_record = []
tanh_lut = tanh(
    input_bit=8,
    dx_bit=8,
    slope_fmt=(10, 10),
    intercept_fmt=(19, 19),
    max=8,
    better_lut=True,
    verbose=False,
    plot=False)

SNR = SNR_list[0]

overall_mu_ber = 0.0
overall_teacher_ber = 0.0
overall_base_ber = 0.0
overall_nopath_ber = 0.0
cnt = 0
packets =[11, 13, 14, 16, 17, 18, 19, 20]  # [11, 13, 14, 16,]
# except 2
def grid_search(i_packet, 
                med_size, 
                input_thres, 
                input_tau_grad, 
                input_scale_grad, 
                thres, 
                tau_grad, 
                scale_grad):
    rc = RC(silent, method, N_total_frame, N_sync_frame, SNR, delay, window_size, i_packet,
            N_reservoir=16,
            spectral_radius=0.2,
            sparsity=0.4,
            noise=1e-6,
            lut_activation=False,  # True,
            tanh_lut=tanh_lut,
            input_scale=25,  #40, #50, # 25,
            reservoir_input_scale = 8,  #4,  #5,
            show_wout=False,
            output_folder= output_folder,
            debug=debug,
            use_fpga= None,
            sock= None,  # usock
            addr = None) # addr

    train_input, train_label, test_input, test_label = rc.run()
    train_input, train_label, test_input, test_label = pre_processing(train_input, train_label)
    train_label = 100 * train_label
    test_label = 100 * test_label
    min_val = np.min(train_input)
    max_val = np.max(train_input)
   # print(min_val, max_val)
    train_data = SpikingDataset(train_input, train_label, sequence_length=nb_inputs, time_step=100, min_val=min_val, max_val=max_val)
    test_data =  SpikingDataset(test_input, test_label, sequence_length=nb_inputs, time_step=100, min_val=min_val, max_val=max_val)
    train_loader = DataLoader(dataset=train_data, batch_size=256, shuffle=False, drop_last=False)
    test_loader = DataLoader(dataset=test_data, batch_size=256, shuffle=False, drop_last=False)
    
    net_snn = Network(4 * nb_inputs, 2, 
                      med_size,
                      input_thres, 
                      input_tau_grad, 
                      input_scale_grad, 
                      thres, 
                      tau_grad,
                      scale_grad).to(device)
    return train(train_loader, test_loader, net_snn, epochs=100, rc=rc, num_frame=19)
global_best_acc = 1e6  
best_params = []
for med_size in [16, 32, 64, 128, 256, 512]:
    for input_thres in [i * 0.1 for i in range(20)]:
        for input_tau_grad in [i * 0.1 for i in range(10)]:
            for input_scale_grad in [10, 1, 0.1, 1e-2, 1e-3]:
                for thres in [i * 0.1 for i in range(1)]:
                    for tau_grad in [i * 0.1 for i in range(10)]:
                        for scale_grad in [10, 1, 0.1, 1e-2, 1e-3]:
                            best_acc = grid_search(11, 
                                         med_size, 
                                         input_thres, 
                                         input_tau_grad, 
                                         input_scale_grad, 
                                         thres, 
                                         tau_grad, 
                                         scale_grad)
                            print(best_acc)
                            if best_acc < global_best_acc:
                                global_best_acc = best_acc
                                best_params = [
                                    med_size,
                                    input_thres, 
                                    input_tau_grad, 
                                    input_scale_grad, 
                                    thres, 
                                    tau_grad,
                                    scale_grad,
                                ]
print("best performance")
print(global_best_acc)
print(best_params)
                                
                                
                             
                            
                        

0.5290570175438597
0.5290570175438597
0.5290570175438597
0.5290570175438597
0.5290570175438597
0.5290570175438597


KeyboardInterrupt: 